<a href="https://colab.research.google.com/github/jbloewencolon/Psychedelic-Trip-Generator/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import csv
import sys
import matplotlib.pyplot as plt

# Specify the path to your CSV file
csv_file = '/content/drive/MyDrive/Colab Notebooks/Data/Full.csv'

# Specify the selected columns
selected_columns = ['title', 'drug', 'dosage', 'delivery', 'weight', 'year', 'gender', 'report']

# Import the CSV file with selected columns
df = pd.read_csv(csv_file, encoding='ISO-8859-1', usecols=selected_columns)

<ipython-input-1-f9242041684a>:13: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file, encoding='ISO-8859-1', usecols=selected_columns)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39257 entries, 0 to 39256
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     39135 non-null  object
 1   drug      39130 non-null  object
 2   dosage    35501 non-null  object
 3   delivery  37855 non-null  object
 4   weight    39128 non-null  object
 5   year      39128 non-null  object
 6   gender    39127 non-null  object
 7   report    39125 non-null  object
dtypes: object(8)
memory usage: 2.4+ MB


In [4]:
df.head()

,title,drug,dosage,delivery,weight,year,gender,report
0,Ode to Joy,MDMA,1.5 tablets,oral,185,2000,male,My friend had some experience with X and had t...
1,Make Sure the Music's Not Too Complex,Cannabis,NaN,smoked,0,1999,not specified,This was the first experience that either my f...
2,After Hours,"MDMA, MDMA, MDMA","160 mg, 100 mg, 50 mg","oral, oral, insufflated",150,2001,male,Preparation: I have heard some conflicting opi...
3,Heavy Hallucinogenic Expreince,"25C-NBOMe, Alcohol - Beer/Wine","1 hit, 2 glasses","sublingual, oral",165,2012,male,00:00 - Just dropped tab alone. Holding under ...
4,Love and the Velvet Underground,DMT,100 mg,smoked,170,2013,male,Setting: my apartmentDate: 4/24/2013I lit up t...


In [8]:
# Checking for missing values in all columns
print(df.isnull().sum())

# If 'column_name' has missing values, fill missing values with 'Unknown'
df.fillna('Unknown', inplace=True)


title        122
drug         127
dosage      3756
delivery    1402
weight       129
year         129
gender       130
report       132
dtype: int64


In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
porter = PorterStemmer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize the words
    words = word_tokenize(text)
    # Remove stopwords and stem the words
    words = [porter.stem(word) for word in words if word not in stop_words]
    # Join words back into a string
    text = ' '.join(words)
    return text

df['processed_report'] = df['report'].apply(preprocess_text)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


NameError: ignored

In [10]:
df.head()


,title,drug,dosage,delivery,weight,year,gender,report,processed_report
0,Ode to Joy,MDMA,1.5 tablets,oral,185,2000,male,My friend had some experience with X and had t...,friend experi x told one day said come across ...
1,Make Sure the Music's Not Too Complex,Cannabis,Unknown,smoked,0,1999,not specified,This was the first experience that either my f...,first experi either friend salvia housem check...
2,After Hours,"MDMA, MDMA, MDMA","160 mg, 100 mg, 50 mg","oral, oral, insufflated",150,2001,male,Preparation: I have heard some conflicting opi...,prepar heard conflict opinion 5htp ie load day...
3,Heavy Hallucinogenic Expreince,"25C-NBOMe, Alcohol - Beer/Wine","1 hit, 2 glasses","sublingual, oral",165,2012,male,00:00 - Just dropped tab alone. Holding under ...,0000 drop tab alon hold tongu wo swallow spit0...
4,Love and the Velvet Underground,DMT,100 mg,smoked,170,2013,male,Setting: my apartmentDate: 4/24/2013I lit up t...,set apartmentd 4242013i lit freebas pipe white...


In [15]:
df['drug'].value_counts()

Salvia divinorum                                           1594
Cannabis                                                   1148
Mushrooms                                                   960
LSD                                                         860
MDMA                                                        849
                                                           ... 
Passion Flower, Cacti - T. pachanoi                           1
GHV                                                           1
Cocaine, Methamphetamine, Cannabis                            1
Alcohol, AMT, AMT                                             1
Clobazam, Clobazam, Clobazam, Clobazam, Pseudoephedrine       1
Name: drug, Length: 11745, dtype: int64

In [17]:
# Strip and lower case the 'drug' column first to avoid duplicates like 'MDMA ' and 'mdma'
df['drug'] = df['drug'].str.lower().str.strip()

# Handle single drug cases
df['drug'] = df['drug'].apply(lambda x: ', '.join(set(x.split(', '))))

# Handle multiple drug cases
# Split the 'drug' column
df['drug'] = df['drug'].str.split(', ')

# Explode/split into multiple rows
df = df.explode('drug')

# Now remove any leading or trailing white space from 'drug' entries
df['drug'] = df['drug'].str.strip()


In [20]:
df['drug'].value_counts()

cannabis              7879
mdma                  2556
lsd                   2463
salvia divinorum      2437
mushrooms             2203
                      ... 
3-me-pce                 1
pipt                     1
6-methoxymethylone       1
lormetazepam             1
methoxyketamine          1
Name: drug, Length: 876, dtype: int64

In [53]:
# Define the substances in each category
other_entactogens = ['mda', 'methylone', '4-methylmethcathinone', '6-apb']
other_dissociatives = ['methoxetamine', 'pcp', '3-meo-pcp', 'deschloroketamine', 'methoxphenidine', 'ephenidine']
nootropics = ['vitamins / supplements', 'piracetam', 'l-theanine', 'ginkgo biloba', 'aniracetam', 'noopept', 'vitamins - choline']
synthetic_cannabinoids = ['products - spice-like smoking blends', 'jwh-018', 'am-2201']
entheogens = ['morning glory', 'syrian rue', 'h.b. woodrose', 'amanitas - a. muscaria', 'mimosa tenuiflora', 'cacti - t. pachanoi', 'banisteriopsis caapi', 'ayahuasca', 'cacti - t. peruvianus', 'anadenanthera colubrina', 'anadenanthera peregrina', 'tabernanthe iboga', 'psychotria viridis', 'acacia confusa', 'voacanga africana', 'harmala alkaloids']
depressants = ['products - spice-like smoking blends', 'ghb', 'alcohol', 'opioids', 'benzodiazepines', 'barbiturates (butalbital)']
anxiolytics = ['etizolam', 'benzodiazepines', 'clonazolam', 'bromazepam', 'diclazepam']
antidepressants = ['"st. johns wort"', 'duloxetine']

# Define function to clean and categorize the drug names
def clean_and_categorize(drug):
    if 'salvia' in drug: return 'salvia'
    elif '3-meo' in drug: return '3-MeO'
    elif '5-meo' in drug: return '5-meo'
    elif '2c' in drug: return '2c'
    elif 'cannabis' in drug: return 'cannabis'
    elif 'mushrooms' in drug: return 'mushrooms'
    elif 'alcohol' in drug: return 'alcohol'
    elif 'tobacco' in drug: return 'tobacco'
    elif drug in ['silene capensis', 'Calea zacatechichi', 'Entada rheedii', 'Mugwort']: return 'oneirogen'
    elif drug == '[]': return 'unknown'
    elif drug in other_entactogens: return 'Other Entactogens'
    elif drug in other_dissociatives: return 'Other Dissociatives'
    elif drug in nootropics: return 'Nootropics'
    elif drug in synthetic_cannabinoids: return 'Synthetic Cannabinoids'
    elif drug in entheogens: return 'Entheogens'
    elif drug in depressants: return 'Depressants'
    elif drug in anxiolytics: return 'Anxiolytics'
    elif drug in antidepressants: return 'Antidepressants'
    else: return drug

# Apply function to clean and categorize drug names
df['drug'] = df['drug'].apply(clean_and_categorize)

# Drop anything with 'pharms'
df = df[~df['drug'].str.contains('pharms')]

# Get the counts of each drug
drug_counts = df['drug'].value_counts()

# Create a list of drugs that appear 10 times or more
drugs_to_keep = drug_counts[drug_counts >= 10].index

# Filter the DataFrame to only include drugs in the above list
df = df[df['drug'].isin(drugs_to_keep)]

In [54]:
# Set maximum rows displayed in output to 300
pd.set_option('display.max_rows', 300)

# Now print the value counts
print(df['drug'].value_counts())

cannabis                                       8280
Sedatives/Depressants                          3432
Entheogens                                     3261
mushrooms                                      3197
Dissociatives                                  2841
mdma                                           2556
lsd                                            2463
Stimulants                                     2344
2c                                             2310
dmt                                            1144
5-meo                                          1042
dxm                                             885
ketamine                                        828
cocaine                                         810
amphetamines                                    783
unknown                                         781
nitrous oxide                                   694
kratom                                          689
Other Entactogens                               596
methamphetam

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60999 entries, 0 to 39256
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             60999 non-null  object
 1   drug              60999 non-null  object
 2   dosage            60999 non-null  object
 3   delivery          60999 non-null  object
 4   weight            60999 non-null  object
 5   year              60999 non-null  object
 6   gender            60999 non-null  object
 7   report            60999 non-null  object
 8   processed_report  60999 non-null  object
dtypes: object(9)
memory usage: 4.7+ MB


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate the TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)

# Fit and transform the processed reports
X = vectorizer.fit_transform(df['processed_report']).toarray()

# Let's say you're going to predict the 'drug' column, convert it to a format suitable for the model.
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(df['drug'])


In [16]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

# Number of topics you want to find
n_topics = 10

# Create and fit the LDA model
lda = LDA(n_components=n_topics)
lda.fit(X)

# Function to print the top words for each topic
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Print the top 10 words per topic
print_top_words(lda, vectorizer.get_feature_names(), 10)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Logistic Regression model
lr = LogisticRegression()

# Train the model
lr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = lr.predict(X_test)

# Print a classification report
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# Create a GridSearchCV object
grid_search = GridSearchCV(lr, param_grid, cv=5)

# Perform grid search
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(grid_search.best_params_)
print(grid_search.best_score_)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Assuming df is your DataFrame and it has 'category' and 'reports' columns
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Text preprocessing
def preprocess_text(text):
    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords and Lemmatize
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

# Apply preprocessing to 'reports' column
df['reports'] = df['reports'].apply(preprocess_text)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['reports'], df['category'], test_size=0.2, random_state=42)

# Convert text data to TF-IDF
vectorizer = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7)
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

# Train the model
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

# Predict categories for test set
predictions = model.predict(X_test)

# Evaluate the model
print(classification_report(y_test, predictions))
